In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# Load the dataset
df = pd.read_csv("combined_output.csv", on_bad_lines='skip')

In [3]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,'5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d...,'kabargayo.com,'Kredit sepeda motor bisa terbayar jika Anda m...,'https://www.kabargayo.com/2024/09/19/kredit-s...,"'Jakarta, VIVA – Pembayaran kredit sepeda moto...",19/09/2024 22.32,'id,19/09/2024 22.32,'Aldi Hadad,'https://i1.wp.com/thumb.viva.co.id/media/fron...,NaN,'positive,NaN,"'Hari Pembayaran Berbayar atau Harcilnas 2024,...",NaN,5250000,Adira,NaN,NaN
1,'e1e3f8d68b58568e8217b7562d48de634fceb0d837135...,'viva.co.id,'Kredit Motor Bisa Lunas Jika Bayar Cicilan Te...,'https://www.viva.co.id/otomotif/tips/1753596-...,"'Jakarta, VIVA – Cicilan kredit motor yang ser...",19/09/2024 22.30,'id,19/09/2024 22.30,'Krisna Wicaksono,'https://thumb.viva.co.id/media/frontend/thumb...,NaN,'positive,NaN,"'Harinya Cicilan Lunas,2024,PT Adira Dinamika ...",NaN,5250000,Adira,NaN,NaN
2,'dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf...,'kabarmegapolitan.pikiran-rakyat.com,'Adira Finance Umumkan Pemenang HARCILNAS 2024...,'https://kabarmegapolitan.pikiran-rakyat.com/b...,'KABARMEGAPOLITAN.com - PT Adira Dinamika Mult...,19/09/2024 21.45,'id,19/09/2024 21.45,'Yuliansyah,'https://assets.pikiran-rakyat.com/www/network...,NaN,'positive,'HARCILNAS merupakan wujud apresiasi kami kepa...,"'PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 ...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,'56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b...,'banggairaya.id,"'Dapatkan Promo Menarik, Yamaha Prima Motor Ra...",'https://banggairaya.id/dapatkan-promo-menarik...,'BANGGAI RAYA- Yamaha Prima Motor ramaikan pam...,19/09/2024 19.45,'id,19/09/2024 19.45,'Chikal Connect,'https://i0.wp.com/banggairaya.id/wp-content/u...,NaN,'neutral,NaN,"'RAYA- Yamaha Prima Motor,Banggai Goverment Ex...",NaN,5250000,Adira,NaN,NaN
4,'1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721...,'jakarta.tribunnews.com,"'Sindikat Penipuan Leasing, Satu Bulan Ajukan ...",'https://jakarta.tribunnews.com/2024/09/19/sin...,'Laporan wartawan TribunJakarta.com Yusuf Bach...,19/09/2024 18.43,'id,19/09/2024 18.43,'Yusuf Bachtiar,'https://asset-2.tstatic.net/jakarta/foto/bank...,NaN,'neutral,'Pelaku ini melakukan pembiayaan pembelian ken...,"'Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [3]:
# Clean extra characters from all columns in the DataFrame
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column is of string type
        df[column] = df[column].str.strip("'")  # Remove extra characters

# Filter relevant columns
df_filtered = df[['body', 'sentiment']].dropna()


In [5]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d4...,kabargayo.com,Kredit sepeda motor bisa terbayar jika Anda me...,https://www.kabargayo.com/2024/09/19/kredit-se...,"Jakarta, VIVA – Pembayaran kredit sepeda motor...",19/09/2024 22.32,id,19/09/2024 22.32,Aldi Hadad,https://i1.wp.com/thumb.viva.co.id/media/front...,NaN,positive,NaN,"Hari Pembayaran Berbayar atau Harcilnas 2024,P...",NaN,5250000,Adira,NaN,NaN
1,e1e3f8d68b58568e8217b7562d48de634fceb0d8371356...,viva.co.id,Kredit Motor Bisa Lunas Jika Bayar Cicilan Tep...,https://www.viva.co.id/otomotif/tips/1753596-k...,"Jakarta, VIVA – Cicilan kredit motor yang seri...",19/09/2024 22.30,id,19/09/2024 22.30,Krisna Wicaksono,https://thumb.viva.co.id/media/frontend/thumbs...,NaN,positive,NaN,"Harinya Cicilan Lunas,2024,PT Adira Dinamika M...",NaN,5250000,Adira,NaN,NaN
2,dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf5...,kabarmegapolitan.pikiran-rakyat.com,Adira Finance Umumkan Pemenang HARCILNAS 2024:...,https://kabarmegapolitan.pikiran-rakyat.com/bi...,KABARMEGAPOLITAN.com - PT Adira Dinamika Multi...,19/09/2024 21.45,id,19/09/2024 21.45,Yuliansyah,https://assets.pikiran-rakyat.com/www/network/...,NaN,positive,HARCILNAS merupakan wujud apresiasi kami kepad...,"PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 p...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b2...,banggairaya.id,"Dapatkan Promo Menarik, Yamaha Prima Motor Ram...",https://banggairaya.id/dapatkan-promo-menarik-...,BANGGAI RAYA- Yamaha Prima Motor ramaikan pame...,19/09/2024 19.45,id,19/09/2024 19.45,Chikal Connect,https://i0.wp.com/banggairaya.id/wp-content/up...,NaN,neutral,NaN,"RAYA- Yamaha Prima Motor,Banggai Goverment Exp...",NaN,5250000,Adira,NaN,NaN
4,1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721d...,jakarta.tribunnews.com,"Sindikat Penipuan Leasing, Satu Bulan Ajukan K...",https://jakarta.tribunnews.com/2024/09/19/sind...,Laporan wartawan TribunJakarta.com Yusuf Bacht...,19/09/2024 18.43,id,19/09/2024 18.43,Yusuf Bachtiar,https://asset-2.tstatic.net/jakarta/foto/bank/...,NaN,neutral,Pelaku ini melakukan pembiayaan pembelian kend...,"Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,T...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [4]:
# Balance dataset classes
class_counts = df_filtered['sentiment'].value_counts()
min_class = class_counts.min()
df_balanced = df_filtered.groupby('sentiment').apply(lambda x: x.sample(min_class)).reset_index(drop=True)

In [5]:
# Split the data into training and validation sets (80% training, 20% validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_filtered['body'].tolist(), df_filtered['sentiment'].tolist(), test_size=0.2, random_state=42
)

In [9]:
# Load the Longformer tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# Tokenize the training and validation texts
# Increase sequence length
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length", max_length=1024)
val_encodings = tokenizer(val_texts, truncation=True, padding="max_length", max_length=1024)


KeyboardInterrupt: 

In [6]:
# Map sentiment labels to integers
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = [label_mapping[label] for label in train_labels]
val_labels = [label_mapping[label] for label in val_labels]

In [9]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [10]:
# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [11]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [12]:
# Load Longformer model for sequence classification
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Adjust model's loss function to include class weights
model.config.problem_type = "single_label_classification"
model.config.class_weights = class_weights.tolist()

In [36]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",            # Evaluate at the end of each epoch
    save_strategy="epoch",            # Save at the end of each epoch
    per_device_train_batch_size=2,    # Small batch size for memory
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,    # Simulate larger batch size
    num_train_epochs=3,               # Fewer epochs for testing
    learning_rate=3e-5,
    weight_decay=0.01,
    fp16=True,                        # Enable mixed precision training
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,      # Load the best model based on eval metric
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    lr_scheduler_type="reduce_lr_on_plateau",
    save_total_limit=2,               # Limit number of saved models
)


In [37]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],  # Increase patience
)

In [38]:
import torch
torch.cuda.empty_cache()


In [39]:
# Train the model
trainer.train()

  0%|          | 10/2700 [00:17<1:16:07,  1.70s/it]

{'loss': 0.661, 'grad_norm': 23.845584869384766, 'learning_rate': 3e-05, 'epoch': 0.01}


  1%|          | 20/2700 [00:34<1:15:29,  1.69s/it]

{'loss': 0.7739, 'grad_norm': 17.706520080566406, 'learning_rate': 3e-05, 'epoch': 0.02}


  1%|          | 30/2700 [00:51<1:15:21,  1.69s/it]

...


100%|██████████| 2700/2700 [1:23:48<00:00,  1.69s/it]

{'eval_loss': 0.7276062965393066, 'eval_runtime': 149.0534, 'eval_samples_per_second': 12.083, 'eval_steps_per_second': 6.045, 'epoch': 3.0}


100%|██████████| 2700/2700 [1:23:49<00:00,  1.86s/it]

{'train_runtime': 5029.7536, 'train_samples_per_second': 4.294, 'train_steps_per_second': 0.537, 'train_loss': 0.5692158008504797, 'epoch': 3.0}


TrainOutput(global_step=2700, training_loss=0.5692158008504797, metrics={'train_runtime': 5029.7536, 'train_samples_per_second': 4.294, 'train_steps_per_second': 0.537, 'total_flos': 1.41880968658944e+16, 'train_loss': 0.5692158008504797, 'epoch': 3.0})

In [40]:
print(f"Number of training examples: {len(train_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")


Number of training examples: 7200
Batch size: 2


In [ ]:
# Save the trained model and tokenizer
model.save_pretrained('./longformer_model2')
tokenizer.save_pretrained('./longformer_model2')

('./longformer_model2\\tokenizer_config.json',
 './longformer_model2\\special_tokens_map.json',
 './longformer_model2\\vocab.json',
 './longformer_model2\\merges.txt',
 './longformer_model2\\added_tokens.json')

In [42]:
# Evaluate the model
results = trainer.evaluate()

100%|██████████| 901/901 [02:37<00:00,  5.71it/s]


In [43]:
print(results)

{'eval_loss': 0.5915368795394897, 'eval_runtime': 158.1829, 'eval_samples_per_second': 11.386, 'eval_steps_per_second': 5.696, 'epoch': 3.0}


In [44]:
# After training, get predictions on the validation dataset
predictions = trainer.predict(val_dataset)

# Get the predicted logits
logits = predictions.predictions

# Convert logits to predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Get the true labels
true_labels = val_labels

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy:.4f}")

100%|██████████| 901/901 [02:35<00:00,  5.81it/s]

Accuracy: 0.7629


In [10]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [12]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model2'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: AKURAT.CO Menteri Pariwisata dan Ekonomi Kreatif (Menparekraf), Sandiaga Uno berharap, kepemimpinan pemerintah baru bisa melanjutkan program pengembangan destinasi wisata nusantara.  "Kebijakan kita terkait pariwisata ini juga adalah inklusif membuka kepada semua pihak untuk dapat berpartisipasi. Jangan sampai 10 tahun sudah dibangun dan di penghujung ini narasi kita malah tidak tersampaikan," kata Sandiaga di Tangerang, Jumat.  Ia mengatakan, selama sisa masa kepemimpinan Kabinet Indonesia Maju (IKM) yang hanya tersisa sekitar beberapa pekan ini, nantinya dapat diteruskan oleh pemerintah baru yang dipimpin langsung Presiden terpilih Prabowo Subianto.  Dia juga mengatakan, di akhir masa jabatannya sebagai Menparekraf bakal fokus pada percepatan penyelesaian pada lima program-program yang telah berjalan sebelumnya.  Adapun program-program unggulan sektor pariwisata yang ditargetkan untuk segera rampung itu di antaranya wisata Danau Toba, 

In [7]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model2'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!


Initializing global attention on CLS token...


Text 1: AKURAT.CO Menteri Pariwisata dan Ekonomi Kreatif (Menparekraf), Sandiaga Uno berharap, kepemimpinan pemerintah baru bisa melanjutkan program pengembangan destinasi wisata nusantara.  "Kebijakan kita terkait pariwisata ini juga adalah inklusif membuka kepada semua pihak untuk dapat berpartisipasi. Jangan sampai 10 tahun sudah dibangun dan di penghujung ini narasi kita malah tidak tersampaikan," kata Sandiaga di Tangerang, Jumat.  Ia mengatakan, selama sisa masa kepemimpinan Kabinet Indonesia Maju (IKM) yang hanya tersisa sekitar beberapa pekan ini, nantinya dapat diteruskan oleh pemerintah baru yang dipimpin langsung Presiden terpilih Prabowo Subianto.  Dia juga mengatakan, di akhir masa jabatannya sebagai Menparekraf bakal fokus pada percepatan penyelesaian pada lima program-program yang telah berjalan sebelumnya.  Adapun program-program unggulan sektor pariwisata yang ditargetkan untuk segera rampung itu di antaranya wisata Danau Toba, Borobudur, Mandalika, Labuan Bajo dan Kup

: 

In [ ]:
# Load the saved model and tokenizer
model = LongformerForSequenceClassification.from_pretrained('./longformer_model').to(device)
tokenizer = LongformerTokenizer.from_pretrained('./longformer_model')

In [ ]:
# Define label mapping
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [ ]:
# Testing new articles
def classify_article(article):
    # Tokenize the input article and move the tensors to the correct device
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Move tokenized inputs to the correct device (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        # Perform inference using the model
        outputs = model(**inputs)
    
    logits = outputs.logits
    # Get the predicted label (index of the highest logit)
    predicted_label = np.argmax(logits.cpu().numpy(), axis=1)[0]
    
    # Calculate sentiment scores (probabilities)
    sentiment_score = torch.softmax(logits, dim=1).cpu().numpy()[0]
    
    # Map predicted label to sentiment text
    sentiment_result = label_mapping[predicted_label]
    
    return sentiment_result, sentiment_score

In [ ]:
# User input for a new article
new_article = input("Your article:")
sentiment_result, sentiment_score = classify_article(new_article)
print(f"The sentiment of the article is: {sentiment_result}")
print(f"Sentiment scores: {sentiment_score}")


The sentiment of the article is: Positive
Sentiment scores: [0.00133929 0.01447496 0.98418576]


In [ ]:
from transformers import LongformerTokenizer, EncoderDecoderModel

# Load summarization model and tokenizer
summarization_model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
summarization_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.bias', 'decoder.roberta.pooler.dense.weight', 'encoder.embeddings.position_ids']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Function to summarize text
def summarize_text(text):
    input_ids = summarization_tokenizer(text, return_tensors="pt", max_length=4096, truncation=True).input_ids
    output_ids = summarization_model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarization_tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [ ]:
# User input for text summarization
article_to_summarize = input("Your article:")

In [ ]:
# Ensure model is moved to the device
summarization_model = summarization_model.to(device)

# Tokenize and summarize
input_ids = summarization_tokenizer(article_to_summarize, return_tensors="pt").input_ids.to(device)
output_ids = summarization_model.generate(input_ids)

# Get the summary from the output tokens
summary = summarization_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print summary
print("Summary of the article:")
print(summary)

Summary of the article:
The program Harcilnas 2024 is based in Jakarta, Indonesia .
The program is aimed at helping the program Harinya Cicilan Lunasan .
Program Harcilas is a program ini e-bike, e-bikes, ebike, hinga, e bike, e/bike .


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Article text to summarize
article_to_summarize = """
Jakarta, VIVA – Pembayaran kredit sepeda motor yang seringkali menjadi beban bagi sebagian besar masyarakat, kini 
dapat diatasi melalui program Hari Pembayaran Berbayar atau Harcilnas 2024 yang diselenggarakan oleh PT Adira Dinamika 
Multi Finance Tbk (Adira Finance). Program ini memungkinkan nasabah melunasi kredit sepeda motor atau kredit lainnya secara 
penuh hanya dengan menukarkan poin yang terkumpul. Pada Harcilnas periode pertama tahun ini, ada 12 nasabah yang berhasil 
melunasi seluruh tagihannya.
"""

# Use "summarize:" prefix
input_text = f"summarize: {article_to_summarize}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)

# Generate the summary with BART's optimized settings
output_ids = summarization_model.generate(
    input_ids,
    max_length=150,            # Shorter summary length
    num_beams=4,               # Beam search for better quality
    early_stopping=True,
)

# Decode the generated summary output
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Summary of the article:")
print(summary)


c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rache\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Summary of the article:
Pembayaran kredit sepeda motor yang seringkali menjadi beban bagi sebagian besar masyarakat. Program ini memungkinkan nasabah melunasi kredit atau kredit lainnya secara penuh.


In [ ]:
# Export results to CSV
import csv

def export_results(sentiment_results, summaries):
    with open('results.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Article', 'Sentiment', 'Sentiment Scores', 'Summary'])
        for article, sentiment, scores, summary in zip([new_article, article_to_summarize], 
                                                        [sentiment_result, 'N/A'], 
                                                        [sentiment_score, 'N/A'], 
                                                        [summary, 'N/A']):
            writer.writerow([article, sentiment, scores.tolist() if isinstance(scores, np.ndarray) else scores, summary])

In [ ]:
# Call the export function
export_results(sentiment_results=[(new_article, sentiment_result, sentiment_score)], 
               summaries=[summary])
print("Results exported to 'results.csv'.")

UnboundLocalError: cannot access local variable 'summary' where it is not associated with a value